In [1]:
import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta

In [51]:
def make_request(uri, max_retries = 5):
    def fire_away(uri):
        response = requests.get(uri)
        assert response.status_code == 200
        return json.loads(response.content)
    current_tries = 1
    while current_tries < max_retries:
        try:
            time.sleep(1)
            response = fire_away(uri)
            return response
        except:
            time.sleep(1)
            current_tries += 1
    return fire_away(uri)

In [52]:
def pull_posts_for(subreddit, start_at, end_at, query, data_type):
    
    def map_posts(posts):
        return list(map(lambda post: {
            'id': post['id'],
            'created_utc': post['created_utc'],
            'prefix': 't4_'
        }, posts))
    
    SIZE = 500
    URI_TEMPLATE = r'https://api.pushshift.io/reddit/search/submission?subreddit={}&after={}&before={}&query={}$data_type={}&size={}'
    
    post_collections = map_posts( \
        make_request( \
            URI_TEMPLATE.format( \
                subreddit, start_at, end_at, query, data_type, SIZE))['data'])
    n = len(post_collections)
    while n == SIZE:
        last = post_collections[-1]
        new_start_at = last['created_utc'] - (10)
        
        more_posts = map_posts( \
            make_request( \
                URI_TEMPLATE.format( \
                    subreddit, new_start_at, end_at, query, data_type, SIZE))['data'])
        
        n = len(more_posts)
        post_collections.extend(more_posts)
    return post_collections

In [56]:
subreddit = 'wallstreetbets'
query = 'CLSN'
data_type = "comment"
end_at = math.ceil(datetime.utcnow().timestamp())
start_at = math.floor((datetime.utcnow() - \
                       timedelta(days=365)).timestamp())
posts = pull_posts_for(subreddit, start_at, end_at, query, data_type)
## ~ 4314
print(len(posts))
## ~ 4306
print(len(np.unique([ post['id'] for post in posts ])))

100
100


In [57]:
def give_me_intervals(start_at, number_of_days_per_interval = 3):
    
    end_at = math.ceil(datetime.utcnow().timestamp())
        
    ## 1 day = 86400,
    period = (86400 * number_of_days_per_interval)
    end = start_at + period
    yield (int(start_at), int(end))
    padding = 1
    while end <= end_at:
        start_at = end + padding
        end = (start_at - padding) + period
        yield int(start_at), int(end)
## test out the solution,
start_at = math.floor(\
     (datetime.utcnow() - timedelta(days=365)).timestamp())
list(give_me_intervals(start_at, 7))

[(1588583145, 1589187945),
 (1589187946, 1589792745),
 (1589792746, 1590397545),
 (1590397546, 1591002345),
 (1591002346, 1591607145),
 (1591607146, 1592211945),
 (1592211946, 1592816745),
 (1592816746, 1593421545),
 (1593421546, 1594026345),
 (1594026346, 1594631145),
 (1594631146, 1595235945),
 (1595235946, 1595840745),
 (1595840746, 1596445545),
 (1596445546, 1597050345),
 (1597050346, 1597655145),
 (1597655146, 1598259945),
 (1598259946, 1598864745),
 (1598864746, 1599469545),
 (1599469546, 1600074345),
 (1600074346, 1600679145),
 (1600679146, 1601283945),
 (1601283946, 1601888745),
 (1601888746, 1602493545),
 (1602493546, 1603098345),
 (1603098346, 1603703145),
 (1603703146, 1604307945),
 (1604307946, 1604912745),
 (1604912746, 1605517545),
 (1605517546, 1606122345),
 (1606122346, 1606727145),
 (1606727146, 1607331945),
 (1607331946, 1607936745),
 (1607936746, 1608541545),
 (1608541546, 1609146345),
 (1609146346, 1609751145),
 (1609751146, 1610355945),
 (1610355946, 1610960745),
 

In [58]:
subreddit = 'wallstreetbets'
query = 'CLSN'
data_type = "comment"
start_at = math.floor(\
    (datetime.utcnow() - timedelta(days=30)).timestamp())
posts = []
for interval in give_me_intervals(start_at, 7):
    pulled_posts = pull_posts_for(
        subreddit, interval[0], interval[1], query, data_type)
    
    posts.extend(pulled_posts)
    time.sleep(.500)
## ~ 4306
print(len(posts))
## ~ 4306
print(len(np.unique([ post['id'] for post in posts ])))

500
500


In [60]:
import praw
import pandas as pd
import datetime as dt
import time

In [61]:
reddit = praw.Reddit(client_id='uH-z4VD2APPyfQ', \
                     client_secret='rueG9i92S3fIV-O2oBY-k8z-f3Ddlw', \
                     user_agent='capstone_test', \
                     username='No_Acanthisitta_368', \
                     password='Red3160!')

Version 7.0.0 of praw is outdated. Version 7.2.0 was released Wednesday February 24, 2021.


In [69]:

#haven't beeen able to get this to finish


## WARNING: REDDIT WILL THROTTLE YOU IF YOU ARE ANNOYING! BE KIND!
TIMEOUT_AFTER_COMMENT_IN_SECS = .350
posts_from_reddit = []
comments_from_reddit = []
for submission_id in np.unique([ post['id'] for post in posts ]):
    submission = reddit.submission(id=submission_id)
    posts_from_reddit.append(submission)
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        comments_from_reddit.append(comment)
        
        if TIMEOUT_AFTER_COMMENT_IN_SECS > 0:
            time.sleep(TIMEOUT_AFTER_COMMENT_IN_SECS)
## ~ 4306
print(len(posts_from_reddit))
## ~ 35216
#print(len(comments_from_reddit)

KeyboardInterrupt: 

In [ ]:
posts_from_reddit